<a href="https://colab.research.google.com/github/zvryab/FYP/blob/main/FYP_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Final Year Project: Enhancing HCI using AI.**

#STEP 1

Main library is LangChain, which provides the framework for connecting the LLM to external tools like databases.

Langchain-core for fundamental objects like prompts and memory, and langchain-community for external integrations, such as PostgreSQL database connectors.

Updated pip, setuptools, and wheel to the latest versions to prevent installation or compatibility issues that could arise when using newer Python packages.

In [ ]:
!pip install langchain #the main langchain framework
!pip install langchain-core #core building blocks of langchain
!pip install langchain-community

!pip install --upgrade pip setuptools wheel

#RUN TERMINAL

You can do this on the bottom left if you have Colab Pro. Else search online for an alternative solution. The solution I had was not very successful and caused around 2-3 weeks delay so prepare for trial and error or pay for that ease of use.

Once terminal is open, run the terminal text, each command is enclosed between the forward dash.

Wait for ollama install, then when llama2 is running, you can pull any model.

llama3.1, 3.2, 3.3 should be free but vary in size.

Terminal Text:

/curl -fsSL https://ollama.com/install.sh | sh /ollama serve & ollama run llama2 /ollama pull llama3.1

#CONNECT LANGCHAIN TO LLM

This code connects the LangChain framework to the local LLM instance hosted via Ollama.
I specify which model to use, here Llama 3.1, but it can be changed to newer versions like 3.3 for better performance if storage allows.

The base_url is the Ngrok link that tunnels from Colab to my local machine.
Setting temperature to 0 ensures that the LLM generates deterministic outputs, which is crucial for reliable SQL query generation, avoiding hallucinations or random variability.

In [ ]:
from langchain_community.llms import Ollama

llm = Ollama( #LLM object inside ollama class
    model="llama3.1",  # or whicever model I pulled, use 3.3 for better results
    base_url="[your ngrok url to connect to Colab goes here]    temperature=0, #controls randomness of LLM, you can refer to the testing section for more information (0= deterministic vs creative if higher such as 0.7)
)


#Memory feature [unavaliable]

Imported ConversationBufferMemory to allow the system to remember previous user queries and system responses during a session.

ConversationChain is then used to link the language model with the memory, enabling multi-turn conversations rather than treating each input as completely independent.

Feature not tested and requires further research.

In [ ]:
from langchain.memory import ConversationBufferMemory #memory management class for storing conversations
from langchain.chains import ConversationChain    #imports a chain class that connects an LLM to a memory system

In [ ]:
# Initialize memory to store conversation context
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

#Database

psycopg2 is a PostgreSQL adaptor for python so it allows code to send SQL commands AND recieve those results.

sqlalchemy is a database tool kit plus ORM (object-relational mapper) to support the database connection in a structured way.

The database connection is then setup and the SQLDatabase(engine) is to allow Langchain to understand the database through almost a structured abstraction so that it has an understanding of the schema/columns to make the queries with that given context.

In [ ]:
!pip install psycopg2 sqlalchemy langchain ollama

In [ ]:
!pip install langchain langchain-community sqlalchemy psycopg2-binary

from sqlalchemy import create_engine #create_engine opens connection to database
from langchain_community.llms import Ollama #connects code to llm hosted via ollama - possible redundancy
from langchain_community.agent_toolkits import create_sql_agent #creates an sql agent which carries out sql queries
from langchain_community.utilities import SQLDatabase #gives langchain the ability to query the database
from langchain.agents.agent_types import AgentType #specifies the type of langchain agent to use

#connects database, all data can be found on ngrok UI on terminal
DB_HOST = "7.tcp.eu.ngrok.io"
DB_PORT = "11662"
DB_NAME = "sampledb"
DB_USER = "postgres"

engine = create_engine(f"postgresql://{DB_USER}@{DB_HOST}:{DB_PORT}/{DB_NAME}") #this actually creates that connection using the inputs from variables above


db = SQLDatabase(engine) #creates a structure abstraction to understand the context of the database

#Prompting

Below are supprotive classes, utilities and components to enhance the prompting feature.

[Note: These were taken from some brief research online and their impact/usage is not 100% defined here so continue further research on their impact]

In [ ]:
from langchain.prompts import PromptTemplate
from langchain.chains.sql_database.prompt import PROMPT_SUFFIX, _mysql_prompt
from langchain.chains import LLMChain

#SQL AGENT

This section builds and connects the core of my system — a prompt-driven SQL generator backed by an LLM agent.

I tested both LangChain's default SQL prompt and a custom-designed prompt tailored to PostgreSQL syntax and behaviour. You can try both and are labelled as 'LangChain PROMPT' and 'CUSTOM PROMPT'

These prompts, combined with the database schema and a conversation aware memory system, were passed to a LangChain agent using the ZERO_SHOT_REACT_DESCRIPTION type. This gives the model flexibility to reason and generate SQL based solely on input and schema, mimicking real conversation flow.

Error-handling and verbosity were enabled to allow debugging and improve model trustworthiness.

In [ ]:

llm = Ollama(model="llama3.1")

#LangChain Prompt
prompt = PromptTemplate(
    input_variables=["input", "table_info"],
    template=_mysql_prompt + PROMPT_SUFFIX, #uses those pre-built langchain sql prompts
)

llm_chain = LLMChain(llm=llm, prompt=prompt, verbose=True) #combines llm and prompt into a pipeline to take inputs

from langchain.prompts import PromptTemplate


#Custom Prompt
custom_sql_prompt = PromptTemplate(   #custom built prompt to add our own specific instructions to ensure accuracy
    input_variables=["input", "table_info"],
    template="""
You are an expert SQL generator designed to interact with a PostgreSQL database.
Generate a syntactically correct SQL query to answer the given natural language question.

Only output the SQL query. Do not include explanations, comments, or intermediate steps.

Schema:
{table_info}

Question:
{input}

SQL Query:
"""
)

#create SQL agent
agent_executor = create_sql_agent(
    llm=llm,
    db=db,
    agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION, #makes decision from a single prompt
    verbose=True,
    handle_parsing_errors=True, #try to recover rathen than crash
    memory=memory
    #prefix= custom_prefix
)


#Interface

We install Gradio as a temporary UI for users to interact with (which includes a shareable link) rather than locally testing on this notebook.

In [ ]:
!pip install gradio

#Data Visualisation

Attempted data visualisation but did not get results so continue this development to see if a solution can be found.

Intended result: The code checks if the data is visualisable (even better with two columns as the tutorial followed supported this) and then outputs it.

[Note: Code is not thoroughly annotated due to limited time spent on development ]

In [ ]:
!pip install matplotlib plotly #plotting library in python for graphs etc

In [ ]:
import matplotlib.pyplot as plt # for bar charts
import pandas as pd # to structure sql results into a dataframe
from sqlalchemy import text #pass raw sql into sqlalchemy safely

def visualize_query(query):
    try:
        # Execute the query and load results into a DataFrame
        with engine.connect() as conn:
            result = conn.execute(text(query))
            data = result.fetchall()
            columns = result.keys()

        # Create a DataFrame from the query result (which is sql)
        df = pd.DataFrame(data, columns=columns)

        # Basic visualization: Bar chart if there are two columns (label and value)
        if df.shape[1] == 2:
            plt.figure(figsize=(8, 5))
            plt.bar(df.iloc[:, 0], df.iloc[:, 1], color='skyblue')
            plt.title("Query Result Visualisation")
            plt.xlabel(df.columns[0])
            plt.ylabel(df.columns[1])
            plt.xticks(rotation=45)
            plt.tight_layout()

            # Save the plot as an image
            plt.savefig("query_result.png")
            plt.close()
            return "query_result.png"
        else:
            return "Visualisation not supported for this query format."
    except Exception as e:
        return f"Error in visualisation: {str(e)}"


#Gradio Continued...

I built a web-based interface using Gradio that enables users to interact with my system in plain language.
Users can choose whether they want a text-based response or a visualised chart.

The interface supports natural language questions, which are translated to SQL by the LangChain agent. The SQL is executed against the database, and the output is either displayed as raw text or a bar chart.

This design improves the accessibility and usability of the system, especially for non-technical users, which aligns with the goal of enhancing human-computer interaction.

In [ ]:
import gradio as gr #builds simple and shareable web interface

def sql_query_agent(input_text, output_type): #this runs everytime a query is sent by a user
    try:

        result = agent_executor.run(input_text)

                # If the output type is "Visualisation", send this to
        if output_type == "Graph":
            image_path = visualize_query(result)
            if image_path.startswith("Error"):
                return image_path
            return gr.Image.update(value=image_path)

        return str(result)
    except Exception as e:
        return f"Error: {str(e)}" #basic error handling to return error string

# Create the Gradio interface
iface = gr.Interface(
    fn=sql_query_agent,  #binds interface to the sql agent
    inputs=[
        gr.Textbox(label="SQL Query"),
        gr.Dropdown(choices=["Text", "Graph"], label="Output Type", value="Text") #lets user choose between text or graph output
    ],
    outputs=gr.Textbox(label="Query Result"),
    title="FYP: SQL Query Agent with Visualisation",
    description="Ask a question about your database, and I will generate and execute the SQL query for you. The agent remembers the context of previous questions.",
)


In [ ]:
iface.launch() #launches the interface